## Transfering knowledge through finetuning

In [1]:
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 6)

Populating the interactive namespace from numpy and matplotlib


## Settings

In [22]:
# Demo mode uses the validation dataset for training, which is smaller and faster to train.
demo = True
log_interval = 100
gpus = 0

# Options are imperative or hybrid. Use hybrid for better performance.
mode = 'hybrid'

# training hyperparameters
batch_size = 256

if demo:
    epochs = 5
    learning_rate = 0.02
    wd = 0.002
else:
    epochs = 40
    learning_rate = 0.05
    wd = 0.002

# the class weight for hotdog class to help the imbalance problem.
positive_class_weight = 5

In [3]:
from __future__ import print_function
import logging
logging.basicConfig(level=logging.INFO)
import os
import time
from collections import OrderedDict
import skimage.io as io

import mxnet as mx
from mxnet.test_utils import download
mx.random.seed(127)

In [11]:
# My data folder path
data_dir = '../../../data/data_hot_dogs'
train_dir = data_dir + '/train'
validation_dir = data_dir + '/validation'

In [9]:
os.listdir(data_dir)

['data_generated_medicotask_v1',
 'data_v2',
 'data_hot_dogs',
 'my_models',
 'data_old',
 'pytorch_models',
 'data_Medico_2018_development_set_v1',
 'data_Medico_2018_development_set_v2',
 'history_of_training',
 'plots',
 '.ipynb_checkpoints']

In [12]:
if not os.path.exists(train_dir):
    os.mkdir(train_dir)
    
if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)

## Files

In [13]:
dataset_files = {'train': ('not_hotdog_train-e6ef27b4.rec', '0aad7e1f16f5fb109b719a414a867bbee6ef27b4'),
                 'validation': ('not_hotdog_validation-c0201740.rec', '723ae5f8a433ed2e2bf729baec6b878ac0201740')}

In [14]:
if demo:
    training_dataset, training_data_hash = dataset_files['validation']
else:
    training_dataset, training_data_hash = dataset_files['train']

validation_dataset, validation_data_hash = dataset_files['validation']

In [16]:
def verified(file_path, sha1hash):
    import hashlib
    sha1 = hashlib.sha1()
    with open(file_path, 'rb') as f:
        while True:
            data = f.read(1048576)
            if not data:
                break
            sha1.update(data)
    matched = sha1.hexdigest() == sha1hash
    if not matched:
        logging.warn('Found hash mismatch in file {}, possibly due to incomplete download.'
                     .format(file_path))
    return matched



In [20]:
url_format = 'https://apache-mxnet.s3-accelerate.amazonaws.com/gluon/dataset/{}'
if not os.path.exists(os.path.join(train_dir,training_dataset)) or not verified(os.path.join(train_dir,training_dataset), training_data_hash):
    logging.info('Downloading training dataset.')
    download(url_format.format(training_dataset),
             overwrite=True, dirname=train_dir)

INFO:root:Downloading training dataset.
INFO:root:downloaded https://apache-mxnet.s3-accelerate.amazonaws.com/gluon/dataset/not_hotdog_validation-c0201740.rec into ../../../data/data_hot_dogs/train/not_hotdog_validation-c0201740.rec successfully


In [21]:
if not os.path.exists(os.path.join(validation_dir,validation_dataset)) or not verified(os.path.join(validation_dir,validation_dataset), validation_data_hash):
    logging.info('Downloading validation dataset.')
    download(url_format.format(validation_dataset),
             overwrite=True, dirname=validation_dir)

INFO:root:Downloading validation dataset.
INFO:root:downloaded https://apache-mxnet.s3-accelerate.amazonaws.com/gluon/dataset/not_hotdog_validation-c0201740.rec into ../../../data/data_hot_dogs/validation/not_hotdog_validation-c0201740.rec successfully


## Iterators

In [23]:
train_iter = mx.io.ImageRecordIter(path_imgrec=os.path.join(train_dir,training_dataset),
                                   min_img_size=256,
                                   data_shape=(3, 224, 224),
                                   rand_crop=True,
                                   shuffle=True,
                                   batch_size=batch_size,
                                   max_random_scale=1.5,
                                   min_random_scale=0.75,
                                   rand_mirror=True)
val_iter = mx.io.ImageRecordIter(path_imgrec=os.path.join(validation_dir,validation_dataset),
                                 min_img_size=256,
                                 data_shape=(3, 224, 224),
                                 batch_size=batch_size)

## Model

## Pulling the pre-trained model

In [24]:
from mxnet.gluon import nn
from mxnet.gluon.model_zoo import vision as models

In [25]:
# get pretrained squeezenet
net = models.squeezenet1_1(pretrained=True, prefix='deep_dog_')
# hot dog happens to be a class in imagenet.
# we can reuse the weight for that class for better performance
# here's the index for that class for later use
imagenet_hotdog_index = 713

Model file is not found. Downloading.


In [27]:
deep_dog_net = models.squeezenet1_1(prefix='deep_dog_', classes=2)

In [29]:
deep_dog_net.collect_params().initialize()

In [50]:
deep_dog_net.features = net.features

## Evaluation - before load pretrained weights

In [52]:
# return metrics string representation
def metric_str(names, accs):
    return ', '.join(['%s=%f'%(name, acc) for name, acc in zip(names, accs)])
metric = mx.metric.create(['acc', 'f1'])

In [53]:
import mxnet.gluon as gluon
from mxnet.image import color_normalize

def evaluate(net, data_iter, ctx):
    data_iter.reset()
    for batch in data_iter:
        data = color_normalize(batch.data[0]/255,
                               mean=mx.nd.array([0.485, 0.456, 0.406]).reshape((1,3,1,1)),
                               std=mx.nd.array([0.229, 0.224, 0.225]).reshape((1,3,1,1)))
        data = gluon.utils.split_and_load(data, ctx_list=ctx, batch_axis=0)
        label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
        outputs = []
        for x in data:
            outputs.append(net(x))
        metric.update(label, outputs)
    out = metric.get()
    metric.reset()
    return out

In [54]:
evaluate(deep_dog_net, val_iter, mx.cpu())

TypeError: object of type 'Context' has no len()

## Reuse class weights